# Introduction

In this example we will use [Gradio](https://www.gradio.app/) to create a simple Web chatbot app. This application will be using the self hosted LLM.

## Prerequisite

Install the required modules: `langchain-community` `langchain` `gradio`

In [ ]:
!python3 -m pip install openai langchain-openai langchain-community langchain gradio

Define the environment parameters:

In [12]:
from getpass import getpass

# Update the BaseURL of the model (we are using the service name)
# $ kubectl get service
base_url = "http://llm/v1"

llm_api_key  = getpass('Input the model API KEY: ')
if not llm_api_key:
    llm_api_key = "dummy"

Input the model API KEY:  ········


Start the ChatBot Web application:

In [13]:
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage
from openai import OpenAI

import gradio as gr


client = OpenAI(
  base_url = base_url,
  api_key = llm_api_key
)

#discover available models and use the first one
available_models = client.models.list()
if len(available_models.data):
    model = available_models.data[0].id
    print(f"Discovered model is: {model}")
else:
    print("No model discovered")
    sys.exit(1)


llm = ChatOpenAI(
    model=model,
    openai_api_key=llm_api_key,
    openai_api_base=base_url,
    # max_tokens=256,
    temperature=0,
)

def stream_response(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
   
    if message is not None:
        history_langchain_format.append(HumanMessage(content=message))
        partial_message = ""
        for response in llm.stream(history_langchain_format):
            partial_message += response.content
            yield partial_message

gr.ChatInterface(stream_response, title="Simple ChatBot", textbox=gr.Textbox(placeholder="What you would like to know?", container=False, scale=7)).launch(share=True)

Discovered model is: meta/llama3-8b-instruct
Running on local URL:  http://127.0.0.1:7873
Running on public URL: https://78523d47f7388a320c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
